In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
import re

# Функция предобработки текста
def preprocess_text(text):
    # Преобразование в нижний регистр
    text = text.lower()
    # Удаление специальных символов
    text = re.sub(r'[^\w\s]', '', text)
    # Удаление цифр
    text = re.sub(r'\d+', '', text)
    # Удаление лишних пробелов
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Создание пайплайна для предобработки и векторизации
text_pipeline = Pipeline([
('preprocess', FunctionTransformer(lambda x: [preprocess_text(text) for text in x])),
('tfidf', TfidfVectorizer(max_features=5000))
])



In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Пример корпуса документов
corpus = [
'Этот документ посвящен машинному обучению.',
'Документ о Python и анализе данных.',
'Машинное обучение с Python очень популярно.',
'Обучение машинного обучения на Python помогает в работе с данными.'
]

# Создание и настройка векторизатора
vectorizer = TfidfVectorizer(
    max_features=100, # Ограничиваем словарь 100 самыми частыми словами
    min_df=2, # Игнорируем слова, которые встречаются менее чем в 2 документах
    max_df=0.95, # Игнорируем слова, которые встречаются более чем в 95% документов
    ngram_range=(1, 2) # Учитываем как отдельные слова, так и пары слов
)

# Преобразование текстов в TF-IDF матрицу
X = vectorizer.fit_transform(corpus)

# Получение имен признаков (терминов)
feature_names = vectorizer.get_feature_names_out()

# Вывод размерности матрицы
print(f"Форма TF-IDF матрицы: {X.shape}")
print(f"Количество уникальных терминов: {len(feature_names)}")

# Просмотр первых 10 терминов
print(f"Первые 10 терминов: {feature_names[:10]}")

# Преобразование разреженной матрицы в массив для просмотра
print("\nTF-IDF значения для первого документа:")
for idx, term in enumerate(feature_names):
    tfidf_value = X[0, idx]
    if tfidf_value > 0:
        print(f"{term}: {tfidf_value:.4f}")



Форма TF-IDF матрицы: (4, 3)
Количество уникальных терминов: 3
Первые 10 терминов: ['python' 'документ' 'обучение']

TF-IDF значения для первого документа:
документ: 1.0000


In [4]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Загрузка данных из набора 20 Newsgroups (уменьшенная версия для демонстрации)
categories = ['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']
newsgroups = fetch_20newsgroups(subset='all', categories=categories, remove=('headers', 'footers', 'quotes'))

X = newsgroups.data
y = newsgroups.target
target_names = newsgroups.target_names

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Настройка TF-IDF векторизатора
tfidf_vectorizer = TfidfVectorizer(
max_features=5000, # Ограничиваем число признаков
min_df=5, # Минимальная частота документа
max_df=0.8, # Максимальная частота документа
ngram_range=(1, 2), # Униграммы и биграммы
stop_words='english' # Удаляем английские стоп-слова
)

# Преобразование текстов в TF-IDF векторы
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Обучение модели логистической регрессии
classifier = LogisticRegression(
    C=10.0, # Параметр регуляризации
    max_iter=1000, # Максимальное число итераций
    solver='liblinear', # Алгоритм оптимизации
    # multi_class='ovr' # One-vs-rest стратегия для мультиклассовой классификации
)

classifier.fit(X_train_tfidf, y_train)

# Прогнозирование на тестовых данных
y_pred = classifier.predict(X_test_tfidf)

# Оценка результатов
print("Отчет о классификации:\n")
print(classification_report(y_test, y_pred, target_names=target_names))

# Анализ наиболее важных признаков для каждого класса
feature_names = tfidf_vectorizer.get_feature_names_out()

def get_top_features_per_class(classifier, feature_names, class_labels, top_n=10):
    """Получение топ-N важнейших признаков для каждого класса"""
    top_features = {}
    for i, class_label in enumerate(class_labels):
        # Получение коэффициентов модели для текущего класса
        if hasattr(classifier, 'coef_'):
            coefficients = classifier.coef_[i]
        else:
            # Для некоторых моделей может потребоваться другой подход
            continue

        # Сортировка признаков по важности
        top_indices = np.argsort(coefficients)[-top_n:]
        top_features[class_label] = [(feature_names[j], coefficients[j]) for j in top_indices]

    return top_features

top_features = get_top_features_per_class(classifier, feature_names, target_names)

# Вывод топ-признаков для каждого класса
for class_label, features in top_features.items():
    print(f"\nТоп-10 признаков для класса '{class_label}':")
for feature, coef in sorted(features, key=lambda x: x[1], reverse=True):
    print(f"{feature}: {coef:.4f}")



ValueError: The 'liblinear' solver does not support multiclass classification (n_classes >= 3). Either use another solver or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.